# Import


In [1]:
import pandas as pd


# Load Data


In [3]:

df = pd.read_csv('../dataset/dataset.csv')



# Small Datapreprocess


In [4]:
# Check data types and missing values before removal
print("Before removing nulls:")
print(df['instruction'].dtype)
print(df['instruction'].isnull().sum())

# Remove rows with null values in 'instruction_augmented' column
df = df.dropna(subset=['instruction'])

# Check data types and missing values after removal
print("\nAfter removing nulls:")
print(df['instruction'].dtype)
print(df['instruction'].isnull().sum())


# Now you can work with the 'data' DataFrame
print(df.head())


Before removing nulls:
object
0

After removing nulls:
object
0
   flags                                        instruction category  \
0      B   question about cancelling order {{Order Number}}    ORDER   
1    BQZ  i have a question about cancelling oorder {{Or...    ORDER   
2   BLQZ    i need help cancelling puchase {{Order Number}}    ORDER   
3     BL         I need to cancel purchase {{Order Number}}    ORDER   
4  BCELN  I cannot afford this order, cancel purchase {{...    ORDER   

         intent                                           response  
0  cancel_order  I've understood you have a question regarding ...  
1  cancel_order  I've been informed that you have a question ab...  
2  cancel_order  I can sense that you're seeking assistance wit...  
3  cancel_order  I understood that you need assistance with can...  
4  cancel_order  I'm sensitive to the fact that you're facing f...  


# Identify Placeholder


for augmented dataset


In [ ]:
import pandas as pd
import re
from collections import Counter

# Detect rows with placeholders
df['has_placeholder'] = df['instruction_augmented'].str.contains(r'\{\{.*?\}\}')

# Count rows with placeholders
rows_with_placeholders = df['has_placeholder'].sum()

# Extract placeholders function
def extract_placeholders(text):
    return re.findall(r'\{\{(.*?)\}\}', text)

# Apply function and get all placeholders
placeholders = df[df['has_placeholder']]['instruction_augmented'].apply(extract_placeholders).explode()

# Count occurrences of each placeholder
placeholder_counts = Counter(placeholders)

# Convert to DataFrame (including frequency count)
placeholder_df = pd.DataFrame(placeholder_counts.items(), columns=['placeholder', 'frequency'])

# Sort by frequency in descending order
placeholder_df = placeholder_df.sort_values(by='frequency', ascending=False)

# Display results
print(f"Rows with placeholders: {rows_with_placeholders}")
print(f"Total unique placeholders: {len(placeholder_df)}\n")

print("Distribution of Placeholders (Frequency Count):")
print(placeholder_df)



for old dataset


instruction only


In [5]:
import pandas as pd
import re
from collections import Counter

# Detect rows with placeholders in 'instruction'
df['has_placeholder'] = df['instruction'].str.contains(r'\{\{.*?\}\}', na=False)

# Count rows with placeholders
rows_with_placeholders = df['has_placeholder'].sum()

# Function to extract placeholders from a given text
def extract_placeholders(text):
    return re.findall(r'\{\{(.*?)\}\}', str(text))

# Apply function to 'instruction' column to get all placeholders
placeholders_instruction = df[df['instruction'].str.contains(r'\{\{.*?\}\}', na=False)]['instruction'].apply(extract_placeholders).explode()

# Count occurrences of each placeholder
placeholder_counts = Counter(placeholders_instruction)

# Convert to DataFrame (including frequency count)
placeholder_df = pd.DataFrame(placeholder_counts.items(), columns=['placeholder', 'frequency'])

# Sort by frequency in descending order
placeholder_df = placeholder_df.sort_values(by='frequency', ascending=False)

# Display results
print(f"Rows with placeholders: {rows_with_placeholders}")
print(f"Total unique placeholders: {len(placeholder_df)}\n")
print("Distribution of Placeholders (Frequency Count):")
print(placeholder_df)

# Optionally, save only the list of unique placeholders (without frequencies)
unique_placeholders = placeholder_df['placeholder'].tolist()
with open('unique_placeholders.txt', 'w') as f:
    f.write("Unique Placeholders:\n")
    for placeholder in unique_placeholders:
        f.write(f"{placeholder}\n")

print("Placeholders saved to 'unique_placeholders.txt'.")


Rows with placeholders: 6670
Total unique placeholders: 9

Distribution of Placeholders (Frequency Count):
        placeholder  frequency
0      Order Number       2907
3      Account Type       1011
2       Person Name        887
4  Account Category        822
8     Refund Amount        624
7   Currency Symbol        372
5     Delivery City        234
6  Delivery Country        177
1    Invoice Number          8
Placeholders saved to 'unique_placeholders.txt'.


both instruction or response


In [ ]:
import pandas as pd
import re
from collections import Counter

# Detect rows with placeholders in either 'instruction' or 'response'
df['has_placeholder'] = (
    df['instruction'].str.contains(r'\{\{.*?\}\}', na=False) | 
    df['response'].str.contains(r'\{\{.*?\}\}', na=False)
)

# Count rows with placeholders
rows_with_placeholders = df['has_placeholder'].sum()

# Function to extract placeholders from a given text
def extract_placeholders(text):
    return re.findall(r'\{\{(.*?)\}\}', str(text))

# Apply function to both 'instruction' and 'response' columns to get all placeholders
placeholders_instruction = df[df['instruction'].str.contains(r'\{\{.*?\}\}', na=False)]['instruction'].apply(extract_placeholders).explode()
placeholders_response = df[df['response'].str.contains(r'\{\{.*?\}\}', na=False)]['response'].apply(extract_placeholders).explode()

# Concatenate placeholders from both columns
placeholders = pd.concat([placeholders_instruction, placeholders_response])

# Count occurrences of each placeholder
placeholder_counts = Counter(placeholders)

# Convert to DataFrame (including frequency count)
placeholder_df = pd.DataFrame(placeholder_counts.items(), columns=['placeholder', 'frequency'])

# Sort by frequency in descending order
placeholder_df = placeholder_df.sort_values(by='frequency', ascending=False)

# Display results
print(f"Rows with placeholders: {rows_with_placeholders}")
print(f"Total unique placeholders: {len(placeholder_df)}\n")
print("Distribution of Placeholders (Frequency Count):")
print(placeholder_df)

# Optionally, save only the list of unique placeholders (without frequencies)
unique_placeholders = placeholder_df['placeholder'].tolist()
with open('unique_placeholders.txt', 'w') as f:
    f.write("Unique Placeholders:\n")
    for placeholder in unique_placeholders:
        f.write(f"{placeholder}\n")

print("Placeholders saved to 'placeholders_with_frequencies.txt' and 'unique_placeholders.txt'.")


Rows with placeholders: 13041
Total unique placeholders: 391

Distribution of Placeholders (Frequency Count):
                           placeholder  frequency
0                         Order Number       8029
3                         Account Type       5440
4                     Account Category       3900
10            Online Order Interaction       2699
12       Customer Support Phone Number       2635
..                                 ...        ...
230                    Forgot PIN code          1
231             Password Recovery Page          1
80   Customer Service Toll-Free Number          1
78                Social Media Handles          1
262                 Account Management          1

[391 rows x 2 columns]
Placeholders saved to 'placeholders_with_frequencies.txt' and 'unique_placeholders.txt'.


# Generate Fake Data For Placeholder and Saved Data


In [6]:

import re
import pandas as pd
from faker import Faker

# Initialize Faker
fake = Faker()

# Define synthetic data generators
data_generators = {
    'order number': lambda: fake.unique.bothify(text='ORD-#######'),
    'invoice number': lambda: fake.unique.bothify(text='INV#####'),
    'person name': lambda: fake.name(),
    'account type': lambda: fake.random_element(elements=(
        'Personal', 'Business', 'Corporate', 'Enterprise', 'VIP',
        'Premium', 'Standard', 'Basic', 'Student', 'Non-Profit', 'Government'
    )),
    'account category': lambda: fake.random_element(elements=(
        'Retail', 'E-commerce', 'Technology', 'Finance', 'Healthcare',
        'Education', 'Real Estate', 'Hospitality', 'Manufacturing',
        'Legal', 'Entertainment', 'Consulting', 'Logistics'
    )),
    'refund amount': lambda: f"${fake.random_number(digits=3)}.00",
    'currency symbol': lambda: fake.currency_symbol(),
    'delivery city': lambda: fake.city(),
    'delivery country': lambda: fake.country()
}

# Function to replace placeholders and track NER labels
def replace_placeholders(text, data_generators):
    if pd.isnull(text):
        return text, []  # Return empty list for NER labels

    placeholder_pattern = re.compile(r'\{\{(.*?)\}\}')  # Match placeholders {{...}}
    placeholders = placeholder_pattern.findall(text)
    ner_labels = []  # Store entity labels

    for placeholder in placeholders:
        placeholder_cleaned = placeholder.strip().lower()  # Normalize case
        if placeholder_cleaned in data_generators:
            replacement_value = data_generators[placeholder_cleaned]()  # Generate synthetic data
            text = text.replace(f'{{{{{placeholder}}}}}', replacement_value, 1)  # Replace placeholder

            # Append entity info in required format
            ner_labels.append({'text': replacement_value, 'label': placeholder_cleaned.replace(" ", "_")})

    return text, ner_labels

# Apply the placeholder replacement function
df[['instruction', 'ner_labels_only']] = df['instruction'].apply(
    lambda x: pd.Series(replace_placeholders(x, data_generators))
)

# Ensure all rows are retained, even if ner_labels is empty
df['ner_labels_only'] = df.apply(lambda row: row['ner_labels_only'] if row['has_placeholder'] else [], axis=1)

print(df.head())

df.to_csv("generated_fake_data.csv", index=False)


   flags                                        instruction category  \
0      B        question about cancelling order ORD-5090958    ORDER   
1    BQZ  i have a question about cancelling oorder ORD-...    ORDER   
2   BLQZ         i need help cancelling puchase ORD-5914872    ORDER   
3     BL              I need to cancel purchase ORD-3943164    ORDER   
4  BCELN  I cannot afford this order, cancel purchase OR...    ORDER   

         intent                                           response  \
0  cancel_order  I've understood you have a question regarding ...   
1  cancel_order  I've been informed that you have a question ab...   
2  cancel_order  I can sense that you're seeking assistance wit...   
3  cancel_order  I understood that you need assistance with can...   
4  cancel_order  I'm sensitive to the fact that you're facing f...   

   has_placeholder                                    ner_labels_only  
0             True  [{'text': 'ORD-5090958', 'label': 'order_numbe...  
1 

# Convert CSV TO JSON AND save FINAL DATA FOR TRAINING


In [7]:
import pandas as pd
import json

# Load the CSV file
df = pd.read_csv("generated_fake_data.csv")

# Ensure the `ner_labels_only` column is properly formatted as JSON
def fix_json_format(value):
    """
    Fix JSON formatting issues in the 'ner_labels_only' column.
    Ensures it is a valid JSON string.
    """
    try:
        # If already valid JSON, return as is
        if isinstance(value, str):
            return json.loads(value.replace("'", "\""))  # Replace single quotes with double quotes
    except json.JSONDecodeError:
        pass  # If there's an error, leave it as is (optional: log the error)
    return []  # Default to empty list if there's an issue

# Apply formatting fix
df["ner_labels_only"] = df["ner_labels_only"].apply(fix_json_format)

# Save as JSON file
df.to_json("generated_fake_data.json", orient="records", indent=4)

print("CSV successfully converted to valid JSON format.")


CSV successfully converted to valid JSON format.
